In [1]:
from pyspark.sql import SparkSession
from delta import *

scala_version = '2.12'  
spark_version = '3.3.0'
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.3.0',
    'io.delta:delta-core_2.12:2.1.1',
    'org.apache.sedona:sedona-spark-shaded-3.4_2.12:1.5.0,'
    'org.datasyslab:geotools-wrapper:1.5.0-28.2'
]
builder = SparkSession.builder\
   .master("local")\
   .appName("kafka-example")\
   .config("spark.jars.packages", ",".join(packages))\
   .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
   .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
   .enableHiveSupport()


spark = builder.getOrCreate()
#spark = configure_spark_with_delta_pip(builder).getOrCreate()


In [2]:
zone_df = (spark.read
           .option("header","True")
           .option("inferSchema","True")
           .csv('taxi_zone_lookup.csv'))

In [3]:
zone_df.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [4]:
zone_df.write.format('delta').mode('overwrite').saveAsTable('taxi_zone_lookup')

In [5]:
%load_ext sparksql_magic

In [6]:
%%sparksql
show tables in default

namespace,tableName,isTemporary
default,device_data_events,False
default,people10m,False
default,taxi_zone_lookup,False
default,taxi_zones,False
